# Spin qubit measurement software

Also see: https://github.com/VandersypenQutech/qtt/

Load packages

In [1]:
import sys, os, tempfile
import numpy as np
%matplotlib inline
%gui qt
import qcodes
import qtt
from qcodes.plots.pyqtgraph import QtPlot
from qcodes.plots.qcmatplotlib import MatPlot

# set data directory
datadir = os.path.join(tempfile.tempdir, 'qdata')
qcodes.DataSet.default_io = qcodes.DiskIO(datadir)    

## Load your station

The model resembles the a 3-dot spin-qubit dot setup. The hardware consists of a virtual
keithley, IVVI racks and a virtual gates object.

In [2]:
import virtualDot
nr_dots = 2
station = virtualDot.initialize(nr_dots=nr_dots)

keithley1 = station.keithley1
keithley3 = station.keithley3

# virtual gates for the model
gates = station.gates

initialized virtual dot system


## Setup measurement windows

We have
* a parameter viewer
* a lot plotting window

In [ ]:
pv = qtt.parameterviewer.createParameterWidget([gates, ])
mwindows = qtt.tools.setupMeasurementWindows(station, create_parameter_widget=False)
plotQ = mwindows['plotwindow']   

## Read out instruments

In [ ]:
print('gate R: %.1f, amplitude: %f' % (gates.R.get(),  keithley3.readnext()) )
snapshotdata = station.snapshot()

## Simple 1D scan loop

We use the `scan1D` function. This function has `scanjob` as argument.

In [ ]:
scanjob = dict({'sweepdata': dict({'param': 'R', 'start': -500, 'end': 1, 'step': .8, 'wait_time': 1e-2}), 'minstrument': [keithley3.amplitude]})
data1d = qtt.scans.scan1D(station, scanjob, location=None, verbose=1)

## Print the scanned data

In [ ]:
print( data1d.default_parameter_name() )
adata = qtt.algorithms.gatesweep.analyseGateSweep(data1d, fig=100)

##  Make a 2D scan

In [ ]:
start=-500
scanjob = dict({'sweepdata': dict({'param': 'R', 'start': start, 'end': start+400, 'step': 4.}), 'minstrument': ['keithley1'], 'wait_time': 0.})
scanjob['stepdata'] = dict({'param': 'L', 'start': start, 'end': start+400, 'step': 1.})
data = qtt.scans.scan2D(station, scanjob, liveplotwindow=plotQ)

## More analysis

In [ ]:
#qtt.scans.plotData(data, fig=30)
pt, resultsfine = qtt.legacy.analyse2dot(data, fig=300, efig=400, istep=1, verbose=2)

## Send data to powerpoint

We can copy data from a dataset to Powerpoint using `qtt.tools.addPPT_dataset(dataset)`.

In [ ]:
_=qtt.tools.addPPT_dataset(data, verbose=1)    

## Show the recorded data

In [6]:
logviewer = qtt.gui.dataviewer.DataViewer(datadir=datadir, verbose=1)

findfilesR: C:\Users\EENDEB~1\AppData\Local\Temp\qdata: 0.0%
DataViewer: found 11 files


In [4]:
from imp import reload ; reload(qtt.gui.dataviewer)

<module 'qtt.gui.dataviewer' from 'c:\\svn\\qtt\\qtt\\gui\\dataviewer.py'>